In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
data=pd.read_csv("Stars.csv")
data

,Temperature,L,R,A_M,Color,Spectral_Class,Type
0,3068,0.002400,0.1700,16.12,Red,M,0
1,3042,0.000500,0.1542,16.60,Red,M,0
2,2600,0.000300,0.1020,18.70,Red,M,0
3,2800,0.000200,0.1600,16.65,Red,M,0
4,1939,0.000138,0.1030,20.06,Red,M,0
...,...,...,...,...,...,...,...
235,38940,374830.000000,1356.0000,-9.93,Blue,O,5
236,30839,834042.000000,1194.0000,-10.63,Blue,O,5
237,8829,537493.000000,1423.0000,-10.73,White,A,5
238,9235,404940.000000,1112.0000,-11.23,White,A,5


In [3]:
Temperature=data['Temperature']
iqr=Temperature.quantile(0.75)-Temperature.quantile(0.25)
d=1.5*iqr
d1=Temperature.quantile(0.25)-d
d2=Temperature.quantile(0.75)+d
print(f"( {d1} , {d2} )")

( -14222.625 , 32622.375 )


In [4]:
print(max(Temperature))

40000


In [5]:
dataNoise =data.loc[data['Temperature']>d2].copy()
AnotherData=data.loc[data['Temperature']<d2].copy()
dataNoise

,Temperature,L,R,A_M,Color,Spectral_Class,Type
30,39000,204000.0,10.60,-4.70,Blue,O,3
49,33750,220000.0,26.00,-6.10,Blue,B,4
99,36108,198000.0,10.20,-4.40,Blue,O,3
100,33300,240000.0,12.00,-6.50,Blue,B,4
101,40000,813000.0,14.00,-6.23,Blue,O,4
109,33421,352000.0,67.00,-5.79,Blue,O,4
159,37800,202900.0,6.86,-4.56,Blue,O,3
214,34190,198200.0,6.39,-4.57,Blue,O,3
231,38234,272830.0,1356.00,-9.29,Blue,O,5
235,38940,374830.0,1356.00,-9.93,Blue,O,5


In [6]:
X=AnotherData[['L','R']]
reg=LinearRegression().fit(X,AnotherData['Temperature'])

In [7]:
predictNoise=reg.predict(dataNoise[['L','R']])
predictNoise

array([12198.98102923, 12467.00855375, 12078.76434074, 12923.94746098,
       24537.36130087, 14993.72081481, 12190.38440594, 12096.78905375,
        8662.24914374, 10730.86369354,  7544.39094696])

In [8]:
data['TemperatureNoise']=data['Temperature'].copy()
data['TemperatureNoise'].loc[data['Temperature']>d2]=predictNoise
data[['TemperatureNoise','Temperature']].loc[data['Temperature']>d2]

C:\Users\Mosp\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,TemperatureNoise,Temperature
30,12198.981029,39000
49,12467.008554,33750
99,12078.764341,36108
100,12923.947461,33300
101,24537.361301,40000
109,14993.720815,33421
159,12190.384406,37800
214,12096.789054,34190
231,8662.249144,38234
235,10730.863694,38940


In [9]:
selectrandom=np.random.randint(0,240,size=10)
selectrandom

array([ 59, 229, 230,  87, 191,  44, 238, 224, 206,  25])

In [10]:
newTemp=data['Temperature'].copy()
newTemp[list(selectrandom)]=np.nan
data['newTemp']=newTemp
data

,Temperature,L,R,A_M,Color,Spectral_Class,Type,TemperatureNoise,newTemp
0,3068,0.002400,0.1700,16.12,Red,M,0,3068.000000,3068.0
1,3042,0.000500,0.1542,16.60,Red,M,0,3042.000000,3042.0
2,2600,0.000300,0.1020,18.70,Red,M,0,2600.000000,2600.0
3,2800,0.000200,0.1600,16.65,Red,M,0,2800.000000,2800.0
4,1939,0.000138,0.1030,20.06,Red,M,0,1939.000000,1939.0
...,...,...,...,...,...,...,...,...,...
235,38940,374830.000000,1356.0000,-9.93,Blue,O,5,10730.863694,38940.0
236,30839,834042.000000,1194.0000,-10.63,Blue,O,5,30839.000000,30839.0
237,8829,537493.000000,1423.0000,-10.73,White,A,5,8829.000000,8829.0
238,9235,404940.000000,1112.0000,-11.23,White,A,5,9235.000000,NaN


In [11]:
data2=data.loc[data['newTemp']==data['Temperature']]

In [12]:
data2

,Temperature,L,R,A_M,Color,Spectral_Class,Type,TemperatureNoise,newTemp
0,3068,0.002400,0.1700,16.12,Red,M,0,3068.000000,3068.0
1,3042,0.000500,0.1542,16.60,Red,M,0,3042.000000,3042.0
2,2600,0.000300,0.1020,18.70,Red,M,0,2600.000000,2600.0
3,2800,0.000200,0.1600,16.65,Red,M,0,2800.000000,2800.0
4,1939,0.000138,0.1030,20.06,Red,M,0,1939.000000,1939.0
...,...,...,...,...,...,...,...,...,...
234,21904,748490.000000,1130.0000,-7.67,Blue-white,B,5,21904.000000,21904.0
235,38940,374830.000000,1356.0000,-9.93,Blue,O,5,10730.863694,38940.0
236,30839,834042.000000,1194.0000,-10.63,Blue,O,5,30839.000000,30839.0
237,8829,537493.000000,1423.0000,-10.73,White,A,5,8829.000000,8829.0


In [13]:
LR=data2[['L','R']]
np.array([np.array(LR['L']),np.array(LR['R'])])
arr=np.array([[x,y]for x,y in zip(np.array(LR['L']),np.array(LR['R']))])
arr

array([[2.40000e-03, 1.70000e-01],
       [5.00000e-04, 1.54200e-01],
       [3.00000e-04, 1.02000e-01],
       [2.00000e-04, 1.60000e-01],
       [1.38000e-04, 1.03000e-01],
       [6.50000e-04, 1.10000e-01],
       [7.30000e-04, 1.27000e-01],
       [4.00000e-04, 9.60000e-02],
       [6.90000e-04, 1.10000e-01],
       [1.80000e-04, 1.30000e-01],
       [2.90000e-03, 5.10000e-01],
       [1.22000e-02, 3.76100e-01],
       [4.00000e-04, 1.96000e-01],
       [5.50000e-03, 3.93000e-01],
       [6.00000e-04, 1.40000e-01],
       [3.80000e-03, 2.40000e-01],
       [1.80000e-03, 1.60000e-01],
       [3.67000e-03, 4.70000e-01],
       [3.62000e-03, 1.96700e-01],
       [3.90000e-02, 3.51000e-01],
       [5.60000e-02, 8.40000e-03],
       [4.90000e-04, 1.23400e-02],
       [1.70000e-04, 1.10000e-02],
       [5.00000e-04, 1.00000e-02],
       [1.30000e-02, 1.40000e-02],
       [8.10000e-04, 9.70000e-03],
       [1.10000e-04, 1.28000e-02],
       [1.50000e-04, 1.10000e-02],
       [8.00000e-05,

In [14]:
reg=LinearRegression().fit(arr,data2['newTemp'])

In [15]:
data4=data.loc[data['newTemp']!=data['Temperature']]
data4

,Temperature,L,R,A_M,Color,Spectral_Class,Type,TemperatureNoise,newTemp
25,12990,0.000085,0.00984,12.230,Yellowish White,F,2,12990.0,NaN
44,3008,280000.000000,25.00000,-6.000,Red,M,4,3008.0,NaN
59,3535,195000.000000,1546.00000,-11.360,Red,M,5,3535.0,NaN
87,10980,0.000740,0.00870,11.190,Blue White,B,2,10980.0,NaN
191,3257,0.002400,0.46000,10.730,Red,M,1,3257.0,NaN
206,24020,0.001590,0.01270,10.550,Blue,B,2,24020.0,NaN
224,16787,246730.000000,62.00000,-6.350,Blue,O,4,16787.0,NaN
229,21738,748890.000000,92.00000,-7.346,Blue,O,4,21738.0,NaN
230,24145,382993.000000,1494.00000,-8.840,Blue-white,B,5,24145.0,NaN
238,9235,404940.000000,1112.00000,-11.230,White,A,5,9235.0,NaN


In [16]:
reg.predict([np.array([174000.00000,1284.0000])])

array([8227.92847872])

In [17]:
data4LR=data4[['L','R']]
arr4=np.array([[x,y]for x,y in zip(np.array(data4LR['L']),np.array(data4LR['R']))])
data4['newTemp']=reg.predict(arr4)

C:\Users\Mosp\AppData\Local\Temp/ipykernel_16256/2809870597.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data4['newTemp']=reg.predict(arr4)


In [18]:
data4

,Temperature,L,R,A_M,Color,Spectral_Class,Type,TemperatureNoise,newTemp
25,12990,0.000085,0.00984,12.230,Yellowish White,F,2,12990.0,8441.957998
44,3008,280000.000000,25.00000,-6.000,Red,M,4,3008.0,16170.627803
59,3535,195000.000000,1546.00000,-11.360,Red,M,5,3535.0,7778.764146
87,10980,0.000740,0.00870,11.190,Blue White,B,2,10980.0,8441.962525
191,3257,0.002400,0.46000,10.730,Red,M,1,3257.0,8440.177650
206,24020,0.001590,0.01270,10.550,Blue,B,2,24020.0,8441.946729
224,16787,246730.000000,62.00000,-6.350,Blue,O,4,16787.0,15094.214717
229,21738,748890.000000,92.00000,-7.346,Blue,O,4,21738.0,29013.638408
230,24145,382993.000000,1494.00000,-8.840,Blue-white,B,5,24145.0,13239.844124
238,9235,404940.000000,1112.00000,-11.230,White,A,5,9235.0,15364.216307


In [19]:
data['newTemp'].loc[data['Temperature']!=data['newTemp']]=data4['newTemp']

C:\Users\Mosp\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [20]:
data[['Temperature','newTemp']].loc[data['Temperature']!=data['newTemp']]

,Temperature,newTemp
25,12990,8441.957998
44,3008,16170.627803
59,3535,7778.764146
87,10980,8441.962525
191,3257,8440.177650
206,24020,8441.946729
224,16787,15094.214717
229,21738,29013.638408
230,24145,13239.844124
238,9235,15364.216307


In [21]:
from scipy.stats import chi2_contingency
from sklearn.preprocessing import LabelEncoder


In [22]:
color=data['Color']
lbl = LabelEncoder()
color=lbl.fit_transform(color)

Spectral_Class=data['Spectral_Class']
lbl = LabelEncoder()
Spectral_Class=lbl.fit_transform(Spectral_Class)


In [23]:
df=(len(np.unique(Spectral_Class))-1)*(len(np.unique(color))-1)
df

96

In [24]:
x_squared, p, dof, expected = chi2_contingency([Spectral_Class,color])
x_squared

568.4638691525676

In [25]:
from scipy.stats import chi2
prob=0.95
critical = chi2.ppf(prob, df)
critical

119.87093929856714